# B-grid `uo` to C-grid

## 1. Load packages

In [1]:
# Ignore warnings
from os import environ
environ["PYTHONWARNINGS"] = "ignore"

In [2]:
# Import makedirs to create directories where I write new files
from os import makedirs

In [3]:
# Load dask
from dask.distributed import Client

# Load intake and cosima cookbook
import intake

# Load xarray for N-dimensional arrays
import xarray as xr

# Load xgcm to interpolate between Arakawa grids
import xgcm

# Load datetime to deal with time formats
import datetime

# Load numpy for numbers!
import numpy as np

# Load xmip for preprocessing (trying to get consistent metadata for making matrices down the road)
from xmip.preprocessing import combined_preprocessing

# Load pandas for DataFrame manipulations
import pandas as pd

## 2. Define some functions

(to avoid too much boilerplate code)

In [4]:
########## functions ##########
print("Defining functions")

def time_window_strings(year_start, num_years, time_type = datetime.datetime):
    """
    return strings for start_time and end_time
    """
    # start_time is first second of year_start
    start_time = time_type(year_start, 1, 1, 0, 0, 0)
    # end_time is last second of last_year
    end_time = time_type(year_start + num_years - 1, 12, 31, 23, 59, 59)

    # Return the weighted average
    return start_time, end_time

def find_latest_version(cat):
    """
    find latest version of selected data
    """
    sorted_versions = cat.df.version.to_list()
    sorted_versions.sort()
    latest_version = sorted_versions[-1]
    return latest_version

def select_latest_cat(cat, **kwargs):
    """
    search latest version of selected data
    """
    selectedcat = cat.search(**kwargs)
    latestselectedcat = selectedcat.search(version=find_latest_version(selectedcat))
    return latestselectedcat

def select_latest_data(cat, xarray_open_kwargs, **kwargs):
    latestselectedcat = select_latest_cat(cat, **kwargs)
    xarray_combine_by_coords_kwargs=dict(
        compat="override",
        data_vars="minimal",
        coords="minimal"
    )
    datadask = latestselectedcat.to_dask(
        xarray_open_kwargs=xarray_open_kwargs,
        xarray_combine_by_coords_kwargs=xarray_combine_by_coords_kwargs,
        parallel=True,
        preprocess=combined_preprocessing,
    )
    return datadask



Defining functions


## 3. Select the model, experiment, ensemble, and time window

In [5]:
# Comment/uncomment
model = "ACCESS-ESM1-5"

In [6]:
# Load catalog
cat_str = "/g/data/dk92/catalog/v2/esm/cmip6-fs38/catalog.json"

In [7]:
# The catalog
cat = intake.open_esm_datastore(cat_str)
cat

,unique
path,1054133
file_type,2
realm,7
frequency,10
table_id,24
project_id,1
institution_id,3
source_id,4
experiment_id,52
member_id,80


In [8]:
experiment = "historical"

In [9]:
ensemble = "r1i1p1f1"

In [10]:
year_start = 1990
num_years = 10

In [11]:
model, experiment, ensemble

('ACCESS-ESM1-5', 'historical', 'r1i1p1f1')

In [12]:
# Check that catalog contains the data requested before creating empty directories
searched_cat = cat.search(
    source_id = model,
    experiment_id = experiment,
    member_id = ensemble,
    variable_id = ["uo", "vo", "mlotst"],
    realm = 'ocean')
np.sort(searched_cat.df.source_id.unique())

array(['ACCESS-ESM1-5'], dtype=object)

In [13]:
########## Start the client and make the `.nc` files ##########
print("Starting client")
client = Client(n_workers=4)#, threads_per_worker=1, memory_limit='16GB') # Note: with 1thread/worker cannot plot thetao. Maybe I need to understand why?
client

Starting client


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 16,Total memory: 125.20 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:40749,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 16
Started: Just now,Total memory: 125.20 GiB
Comm: tcp://127.0.0.1:33679,Total threads: 4
Dashboard: http://127.0.0.1:39709/status,Memory: 31.30 GiB
Nanny: tcp://127.0.0.1:44001,


In [14]:
# uo dataset
print("Loading uo data")
uo_datadask = select_latest_data(searched_cat,
    dict(
        chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
    ),
    variable_id = "uo",
    frequency = "mon",
)
# print("\nuo_datadask: ", uo_datadask)
uo_datadask

Loading uo data


<xarray.Dataset>
Dimensions:        (time: 1980, lev: 50, y: 300, x: 360, bnds: 2, vertex: 4)
Coordinates:
  * time           (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T1...
    time_bounds    (time, bnds) datetime64[ns] dask.array<chunksize=(120, 2), meta=np.ndarray>
  * lev            (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
    lev_bounds     (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat            (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lon            (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lat_verticies  (y, x, vertex) float64 dask.array<chunksize=(60, 60, 4), meta=np.ndarray>
    lon_verticies  (y, x, vertex) float64 dask.array<chunksize=(60, 60, 4), meta=np.ndarray>
  * y              (y) int64 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    lon_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    uo             (time, lev, y, x) float32 dask.array<chunksize=(120, 50, 60, 60), meta=np.ndarray>
Attributes: (12/58)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            21915.0
    data_specs_version:               01.00.30
    ...                               ...
    intake_esm_attrs:member_id:       r1i1p1f1
    intake_esm_attrs:variable_id:     uo
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20191115
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r...

In [15]:
# vo dataset
print("Loading vo data")
vo_datadask = select_latest_data(searched_cat,
    dict(
        chunks={'i': 60, 'j': 60, 'time': -1, 'lev':50}
    ),
    variable_id = "vo",
    frequency = "mon",
)
# print("\nvo_datadask: ", vo_datadask)
vo_datadask

Loading vo data


<xarray.Dataset>
Dimensions:        (time: 1980, lev: 50, y: 300, x: 360, bnds: 2, vertex: 4)
Coordinates:
  * time           (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T1...
    time_bounds    (time, bnds) datetime64[ns] dask.array<chunksize=(120, 2), meta=np.ndarray>
  * lev            (lev) float64 5.0 15.0 25.0 ... 5.166e+03 5.499e+03 5.831e+03
    lev_bounds     (lev, bnds) float64 dask.array<chunksize=(50, 2), meta=np.ndarray>
    lat            (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lon            (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lat_verticies  (y, x, vertex) float64 dask.array<chunksize=(60, 60, 4), meta=np.ndarray>
    lon_verticies  (y, x, vertex) float64 dask.array<chunksize=(60, 60, 4), meta=np.ndarray>
  * y              (y) int64 0 1 2 3 4 5 6 7 ... 292 293 294 295 296 297 298 299
  * x              (x) int64 0 1 2 3 4 5 6 7 ... 352 353 354 355 356 357 358 359
    lon_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
    lat_bounds     (bnds, y, x) float64 dask.array<chunksize=(1, 60, 60), meta=np.ndarray>
Dimensions without coordinates: bnds, vertex
Data variables:
    vo             (time, lev, y, x) float32 dask.array<chunksize=(120, 50, 60, 60), meta=np.ndarray>
Attributes: (12/58)
    Conventions:                      CF-1.7 CMIP-6.2
    activity_id:                      CMIP
    branch_method:                    standard
    branch_time_in_child:             0.0
    branch_time_in_parent:            21915.0
    data_specs_version:               01.00.30
    ...                               ...
    intake_esm_attrs:member_id:       r1i1p1f1
    intake_esm_attrs:variable_id:     vo
    intake_esm_attrs:grid_label:      gn
    intake_esm_attrs:version:         v20191115
    intake_esm_attrs:_data_format_:   netcdf
    intake_esm_dataset_key:           l.CMIP.CSIRO.ACCESS-ESM1-5.historical.r...

In [16]:
start_time, end_time = time_window_strings(year_start, num_years) # these are not strings (anymore)... TODO: rename this function
start_time, end_time

(datetime.datetime(1990, 1, 1, 0, 0),
 datetime.datetime(1999, 12, 31, 23, 59, 59))

In [17]:
# Slice uo dataset for the time period
uo_datadask_sel = uo_datadask.sel(time=slice(start_time, end_time))
# Take the time average of the monthly evaporation (using month length as weights)
uo = uo_datadask_sel["uo"].weighted(uo_datadask_sel.time.dt.days_in_month).mean(dim="time")
uo

<xarray.DataArray 'uo' (lev: 50, y: 300, x: 360)>
dask.array<truediv, shape=(50, 300, 360), dtype=float64, chunksize=(50, 60, 60), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 5.0 15.0 25.0 35.0 ... 5.166e+03 5.499e+03 5.831e+03
    lat      (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 291 292 293 294 295 296 297 298 299
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359

In [18]:
# Slice vmo dataset for the time period
vo_datadask_sel = vo_datadask.sel(time=slice(start_time, end_time))
# Take the time average of the monthly evaporation (using month length as weights)
vo = vo_datadask_sel["vo"].weighted(vo_datadask_sel.time.dt.days_in_month).mean(dim="time")
vo

<xarray.DataArray 'vo' (lev: 50, y: 300, x: 360)>
dask.array<truediv, shape=(50, 300, 360), dtype=float64, chunksize=(50, 60, 60), chunktype=numpy.ndarray>
Coordinates:
  * lev      (lev) float64 5.0 15.0 25.0 35.0 ... 5.166e+03 5.499e+03 5.831e+03
    lat      (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
    lon      (y, x) float64 dask.array<chunksize=(60, 60), meta=np.ndarray>
  * y        (y) int64 0 1 2 3 4 5 6 7 8 ... 291 292 293 294 295 296 297 298 299
  * x        (x) int64 0 1 2 3 4 5 6 7 8 ... 351 352 353 354 355 356 357 358 359

## Now use xgcm to interpolate `uo` and `vo` to the corresponding C-grid